In [1]:
from model import *
from data import *
import cv2

Using TensorFlow backend.


In [ ]:
model_checkpoint = ModelCheckpoint('unet_weights.hdf5', monitor='loss',verbose=1, save_best_only=True)
model = unet()

## Train dengan data Generator

In [ ]:
data_gen_args = dict(rotation_range=0.2,
                    width_shift_range=0.05,
                    height_shift_range=0.05,
                    shear_range=0.05,
                    zoom_range=0.05,
                    horizontal_flip=True,
                    fill_mode='nearest')
myGene = trainGenerator(2,'data/train','image','label',data_gen_args,save_to_dir = None)
model.fit_generator(myGene, steps_per_epoch = 2000, epochs = 5, callbacks = [model_checkpoint])

## Train dengan npy file

In [ ]:
imgs_train, imgs_mask_train = geneTrainNpy("data/train/aug/","data/train/aug/")
model.fit(imgs_train, imgs_mask_train, batch_size = 2, epochs = 10, verbose = 1, validation_split= 0.2, shuffle = True, callbacks = [model_checkpoint])

## Simpan Data Prediksi

In [ ]:
testGene = testGenerator('data/test')
results = cv2.normalize(src=model.predict_generator(testGene,30,verbose=1), dst=None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)
saveResult("data/test", results)